Filtrado de spam en mensajes de texto (SMS) usando sckkit-learn
===

**Juan David Velásquez Henao**  
jdvelasq@unal.edu.co   
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia

---

Haga click [aquí](https://github.com/jdvelasq/IPython-for-predictive-analytics/blob/master/03-NaiveBayes-SMS-spam.ipynb) para acceder a la última versión online.

Haga click [aquí](http://nbviewer.jupyter.org/github/jdvelasq/IPython-for-predictive-analytics/blob/master/03-
NaiveBayes-SMS-spam.ipynb) para ver la última versión online en `nbviewer`. 

---
[Licencia](https://github.com/jdvelasq/IPython-for-predictive-analytics/blob/master/LICENSE)  
[Readme](https://github.com/jdvelasq/IPython-for-predictive-analytics/blob/master/readme.md)

In [1]:
## Lee el archivo. La función readlines() retorna una 
## una lista de strings donde cada string es una linea
## del archivo original.
sms_raw = open('data/sms_spam.csv').readlines()

## Elimina las comillas dobles al principio y al final
lines = []
for line in sms_raw:
    lines.append(line[1:-2]) 
sms_raw = lines

## Convierte cada linea en una lista de strings, 
## partiendo la línea original por las comas. 
sms_raw = [x.split('","') for x in sms_raw]

## Elimina la primera fila que corresponde a los
## encabezamientos
sms_raw = sms_raw[1:]

## Imprime los primeros cinco registros
sms_raw[0:5]

[['ham',
  'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'],
 ['ham', 'Ok lar... Joking wif u oni...'],
 ['spam',
  "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"],
 ['ham', 'U dun say so early hor... U c already then say...'],
 ['ham', "Nah I don't think he goes to usf, he lives around here though"]]

In [2]:
## Separa el texto y el tipo de mensaje
sms_raw_type = [x[0] for x in sms_raw]
sms_raw_text = [x[1] for x in sms_raw]

In [3]:
## Se crea una función que mimifica 
## la función table() de R
def table(x, y=None):
    if y is None:
        return {a:x.count(a) for a in set(x)}
    return {a:{b:a.count(b) for b in a} for a in set(x)}

In [4]:
## Cuenta la cantidad de ham y spam
table(sms_raw_type)

{'ham': 4827, 'spam': 747}

In [5]:
def prop_table(x):
    L = len(x)
    return {y:x.count(y)/L for y in set(x)}

In [6]:
## Se convierte el conteo en probabilidades.
prop_table(sms_raw_type)

{'ham': 0.8659849300322928, 'spam': 0.1340150699677072}

In [7]:
## Matriz de términos del documento
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

count_vect = CountVectorizer(analyzer='word',        # a nivel de palabra
                             lowercase=True,         # convierte a minúsculas
                             stop_words='english',   # stop_words en inglés
                             min_df=7)               # ignora palabras con baja freq


## Aplica la función al texto
sms_dtm = count_vect.fit_transform(sms_raw_text)

## Las filas contienen los mensajes
## y las clomunas los términos
sms_dtm.shape

(5574, 1187)

In [8]:
## Palabras aprendidas de los mensajes de texto
vocabulary = count_vect.get_feature_names()
len(vocabulary)

1187

In [9]:
vocabulary[0:10]

['00',
 '000',
 '02',
 '03',
 '04',
 '06',
 '0800',
 '08000839402',
 '08000930705',
 '0870']

In [10]:
def dtm2words(dtm, vocabulary, index):
    as_list = dtm[index,:].toarray().tolist()
    docs = []
    for i in index:
        k = [vocabulary[iword] for iword, ifreq in enumerate(as_list[i]) if ifreq > 0]
        docs += [k]
    return docs
    
dtm2words(sms_dtm, vocabulary, [0, 1, 2, 3])

[['available',
  'bugis',
  'cine',
  'crazy',
  'got',
  'great',
  'la',
  'point',
  'wat',
  'world'],
 ['lar', 'ok', 'wif'],
 ['apply',
  'comp',
  'cup',
  'entry',
  'final',
  'free',
  'question',
  'rate',
  'receive',
  'std',
  'text',
  'txt',
  'win',
  'wkly'],
 ['dun', 'early', 'say']]

In [11]:
## Convierte la frecuencia a 0s y 1s
_sms_dtm = sms_dtm.toarray().tolist()
_sms_dtm = [[min(y, 1) for y in x] for x in _sms_dtm]

#s = sms_dtm.data[sms_dtm.data > 1]
#s
#sms_dtm.shape

In [12]:
## Creación de los conjuntos de entrenamiento y prueba.
sms_dtm_train    = _sms_dtm[0:4168 ]
sms_dtm_test     = _sms_dtm[4169:]
sms_train_labels = sms_raw_type[0:4168]
sms_test_labels  = sms_raw_type[4169:]

In [13]:
## Distribución del tipo de mensaje en el 
## conjunto de entrenamiento
prop_table(sms_train_labels)

{'ham': 0.8646833013435701, 'spam': 0.13531669865642995}

In [14]:
## Distribución del tipo de mensaje en el 
## conjunto de prueba
prop_table(sms_test_labels)

{'ham': 0.8697508896797153, 'spam': 0.1302491103202847}

In [15]:
## Entrena el modelo
from sklearn.neighbors import KNeighborsClassifier
kNN = KNeighborsClassifier(n_neighbors=21)
kNN.fit(sms_dtm_train, sms_train_labels)
kNN

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=21, p=2,
           weights='uniform')

In [ ]:
## Se pronostica para los datos de prueba.
sms_test_pred = kNN.predict(sms_dtm_test)
sms_test_pred

In [ ]:
## Métricas de desempeño
##
## Note que en R la respuesta fue:
##
##                  sms_test_pred
##   sms_test_labels  ham spam
##              ham  1200    9
##             spam    20  161
##
## ¿puede explicar por qué?
##
from sklearn.metrics import confusion_matrix
confusion_matrix(y_true = sms_test_labels, 
                 y_pred = sms_test_pred)

# Ejercicio.-- Filtrado de spam en correos electrónicos

### Definición del problema real

Con el crecimiento exponencial del uso de internet para actividades económicas y de ocio, también ha crecido su uso como medio para envio de publicidad no deseada a través del correo electrónico. En este problema en particular, se desea tener una herramienta que permita la clasificación automática de correos con el fin de integrarla posteriormente a un servidor de correo electrónico.   

### Definición del problema en términos de los datos

Se desea construir un sistema que clasifique de forma automática los correos electrónicos como `ham` (genuinos) o `spam`. La carpeta `data/spam-filter` contiene ejemplos de correos electrónicos tanto validos como spam. Dentro de dicho directorio hay tres carpetas con los ejemplos:


* spam: correos spam.


* easy_ham: correos válidos fácilmente diferenciables de correos spam.


* hard_spam: correos válidos más dificiles de diferenciar de correos spam.


Estos tres directorios contienen los correos que deben ser usados para el entrenamiento del clasificador. Los nombres de los directorios terminados en `_2` deben usarse para probar el desempeño del clasificador.

### Requerimientos

Construya clasificadores que usen las siguientes técnicas y comparelos:


* kNN.


* Naive Bayes.

---

Filtrado de spam en mensajes de texto (SMS) usando sckkit-learn
===

**Juan David Velásquez Henao**  
jdvelasq@unal.edu.co   
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia

---

Haga click [aquí](https://github.com/jdvelasq/IPython-for-predictive-analytics/blob/master/03-NaiveBayes-SMS-spam.ipynb) para acceder a la última versión online.

Haga click [aquí](http://nbviewer.jupyter.org/github/jdvelasq/IPython-for-predictive-analytics/blob/master/03-
NaiveBayes-SMS-spam.ipynb) para ver la última versión online en `nbviewer`. 

---
[Licencia](https://github.com/jdvelasq/IPython-for-predictive-analytics/blob/master/LICENSE)  
[Readme](https://github.com/jdvelasq/IPython-for-predictive-analytics/blob/master/readme.md)